In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt


In [2]:
train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

In [3]:
train.shape, test.shape

((1460, 81), (1459, 80))

In [4]:
train_id = train['Id']
test_id = test['Id']

train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

In [5]:
cat_features = train.select_dtypes(include = object).columns
print (cat_features)
num_features = train.select_dtypes(exclude = object).columns
print (num_features)

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')
Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'Kitch

In [6]:
cat_cols = train.select_dtypes(include = object)
num_cols = train.select_dtypes(exclude = object)

In [7]:
print(len(cat_features), len(num_features))

43 37


In [8]:
# import seaborn as sns
# sns.pairplot(num_cols)

In [9]:
cat_cols.describe()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,91,1460,1460,1460,1460,1460,1460,1460,...,1379,1379,1379,1379,1460,7,281,54,1460,1460
unique,5,2,2,4,4,2,5,3,25,9,...,6,3,5,5,3,3,4,4,9,6
top,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal
freq,1151,1454,50,925,1311,1459,1052,1382,225,1260,...,870,605,1311,1326,1340,3,157,49,1267,1198


In [10]:
for i in cat_features:
    train[cat_features]=train[cat_features].fillna(train[cat_features].mode().iloc[0])

In [11]:
for i in num_features:
    train[num_features]=train[num_features].fillna(train[num_features].median().iloc[0])

In [12]:
cat_fs = train.select_dtypes(include = object).columns
num_fs = test.select_dtypes(exclude = object).columns

In [13]:
for i in cat_fs:
    test[cat_fs]=test[cat_fs].fillna(test[cat_fs].mode().iloc[0])

In [14]:
for i in num_fs:
    test[num_fs]=test[num_fs].fillna(test[num_fs].mode().iloc[0])

In [15]:
#list(train.columns) + list(train.dtypes) + list(train.apply(pd.Series.nunique))
list(zip(list(zip(list(train.columns),list(train.dtypes))),list(train.apply(pd.Series.nunique))))

[(('MSSubClass', dtype('int64')), 15),
 (('MSZoning', dtype('O')), 5),
 (('LotFrontage', dtype('float64')), 110),
 (('LotArea', dtype('int64')), 1073),
 (('Street', dtype('O')), 2),
 (('Alley', dtype('O')), 2),
 (('LotShape', dtype('O')), 4),
 (('LandContour', dtype('O')), 4),
 (('Utilities', dtype('O')), 2),
 (('LotConfig', dtype('O')), 5),
 (('LandSlope', dtype('O')), 3),
 (('Neighborhood', dtype('O')), 25),
 (('Condition1', dtype('O')), 9),
 (('Condition2', dtype('O')), 8),
 (('BldgType', dtype('O')), 5),
 (('HouseStyle', dtype('O')), 8),
 (('OverallQual', dtype('int64')), 10),
 (('OverallCond', dtype('int64')), 9),
 (('YearBuilt', dtype('int64')), 112),
 (('YearRemodAdd', dtype('int64')), 61),
 (('RoofStyle', dtype('O')), 6),
 (('RoofMatl', dtype('O')), 8),
 (('Exterior1st', dtype('O')), 15),
 (('Exterior2nd', dtype('O')), 16),
 (('MasVnrType', dtype('O')), 4),
 (('MasVnrArea', dtype('float64')), 327),
 (('ExterQual', dtype('O')), 4),
 (('ExterCond', dtype('O')), 5),
 (('Foundation

In [16]:
#Convert the below fields from numerical to categorical
train['MSSubClass']=train['MSSubClass'].astype(str)
train['MoSold']=train['MoSold'].astype(str)
train['OverallQual']=train['OverallQual'].astype(str)
train['OverallCond']=train['OverallCond'].astype(str)

In [17]:
test['MSSubClass']=test['MSSubClass'].astype(str)
test['MoSold']=test['MoSold'].astype(str)
test['OverallQual']=test['OverallQual'].astype(str)
test['OverallCond']=test['OverallCond'].astype(str)

In [18]:
houses=pd.concat([train,test], sort=False)
len_train=train.shape[0]
#houses['LotFrontage']=houses['LotFrontage'].astype(float)

In [19]:
numerical = houses.dtypes[houses.dtypes != 'object'].index
from scipy.stats import skew 
skewness=houses[numerical].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness_df = pd.DataFrame({'Skewness': skewness})
skewness_df.head(10)

,Skewness
MiscVal,21.947195
PoolArea,16.898328
LotArea,12.822431
LowQualFinSF,12.088761
3SsnPorch,11.376065
KitchenAbvGr,4.302254
BsmtFinSF2,4.146143
EnclosedPorch,4.003891
ScreenPorch,3.946694
BsmtHalfBath,3.931594


In [20]:
(abs(skewness) > 0.50).value_counts()

True     27
False     6
dtype: int64

In [21]:
from scipy.special import boxcox1p
boxcox_features = skewness_df[np.abs(skewness_df['Skewness'])>0.50].index
lam = 0.10

for col in boxcox_features:
    houses.loc[:, col] = boxcox1p(houses[col], lam)

In [22]:
skewness2 = houses[numerical].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)

skewness_df2 = pd.DataFrame({'Skewness': skewness2})
skewness_df2.head(10)

,Skewness
PoolArea,15.071918
3SsnPorch,8.885563
LowQualFinSF,8.671340
MiscVal,5.422529
BsmtHalfBath,3.783479
KitchenAbvGr,3.642872
ScreenPorch,2.965344
BsmtFinSF2,2.524526
EnclosedPorch,2.000413
MasVnrArea,0.588597


In [23]:
houses.shape, train.shape, test.shape

((2919, 80), (1460, 80), (1459, 79))

In [24]:
dummy_houses = pd.get_dummies(houses)

In [25]:
train = dummy_houses[:len_train]
test = dummy_houses[len_train:]
train.shape, test.shape

((1460, 332), (1459, 332))

In [26]:
test.drop("SalePrice", axis = 1, inplace = True)
train.shape, test.shape

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


((1460, 332), (1459, 331))

In [27]:
y_train = train['SalePrice'].values
train.drop("SalePrice", axis = 1, inplace = True)

In [28]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split, KFold
kfold = KFold(n_splits=10)

In [29]:
ridge_results = cross_val_score(RidgeCV(), train.values, y_train, cv=kfold, scoring= 'neg_mean_squared_error')
lasso_results = cross_val_score(LassoCV(), train.values, y_train, cv=kfold, scoring= 'neg_mean_squared_error')
rf_30_results = cross_val_score(RandomForestRegressor(n_estimators=30), train.values, y_train, cv=kfold, scoring= 'neg_mean_squared_error')
rf_100_results = cross_val_score(RandomForestRegressor(n_estimators=100), train.values, y_train, cv=kfold, scoring= 'neg_mean_squared_error')

print ('RMSE for Ridge regression is %.3f\n' % np.sqrt(np.abs(ridge_results.mean())))
print ('RMSE for Lasso regression is %.3f\n' % np.sqrt(np.abs(lasso_results.mean())))
print ('RMSE for Random Forest regression with 30 trees is %.3f\n' % np.sqrt(np.abs(rf_30_results.mean())))
print ('RMSE for Random Forest regression with 100 trees  is %.3f' % np.sqrt(np.abs(rf_100_results.mean())))

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt

RMSE for Ridge regression is 0.425

RMSE for Lasso regression is 0.715

RMSE for Random Forest regression with 30 trees is 0.496

RMSE for Random Forest regression with 100 trees  is 0.480


In [30]:
sc=RobustScaler()
train=sc.fit_transform(train)
test=sc.transform(test)

In [31]:
model=Lasso(alpha =0.001, random_state=1)

In [32]:
model.fit(train,y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=1,
      selection='cyclic', tol=0.0001, warm_start=False)

In [33]:
dict(zip(houses.columns, model.coef_))

{'MSSubClass': -0.0,
 'MSZoning': 0.096779972115293,
 'LotFrontage': 0.23867001887091602,
 'LotArea': 0.10211017582078971,
 'Street': 0.0040237495532064295,
 'Alley': 0.15691992849157446,
 'LotShape': -0.0013110105297691587,
 'LandContour': -0.026517474328026307,
 'Utilities': 0.04802141513653897,
 'LotConfig': 0.053713469950628354,
 'LandSlope': -0.0,
 'Neighborhood': -0.015878092297375225,
 'Condition1': 0.6357269176120448,
 'Condition2': 0.07498801493775537,
 'BldgType': 0.0011181404434251357,
 'HouseStyle': 0.1043590637140814,
 'OverallQual': 0.057553491447848736,
 'OverallCond': -0.006859854875031472,
 'YearBuilt': -0.32511930352035284,
 'YearRemodAdd': 0.005629957083208341,
 'RoofStyle': 0.0970339685413775,
 'RoofMatl': 0.00024946148390147523,
 'Exterior1st': 0.12525551321098466,
 'Exterior2nd': 0.03762863361352611,
 'MasVnrType': 0.06966501951255614,
 'MasVnrArea': 0.02943592632756324,
 'ExterQual': 0.008824126336911587,
 'ExterCond': 0.0107295770240573,
 'Foundation': 0.0230461

In [34]:
pred=model.predict(test)
preds=np.exp(pred)

In [35]:
output=pd.DataFrame({'Id':test_id, 'SalePrice':preds})
output.to_csv('submission.csv', index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 